In [1]:
import numpy as np
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect("D:\\PORTFOLIO PROJECTS\\New folder\\database.sqlite")


In [3]:
c = conn.cursor()

In [4]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = c.fetchall()
# Print the table names
for table in tables:
    print(table[0])

Player
Extra_Runs
Batsman_Scored
Batting_Style
Bowling_Style
Country
Season
City
Outcome
Win_By
Wicket_Taken
Venue
Extra_Type
Out_Type
Toss_Decision
Umpire
Team
Ball_by_Ball
sysdiagrams
sqlite_sequence
Match
Rolee
Player_Match


###### Solving business questions for the database of IPL Sports Analysis

In [5]:
#1.Number of matches played in each stadium
sql=""" SELECT ve.Venue_Id, ve.Venue_Name, COUNT(ma.Outcome_type) AS No_of_match_played, ci.City_Name AS Location
FROM Venue ve
JOIN Match ma ON ve.Venue_Id = ma.Venue_Id
JOIN City ci ON ve.City_Id = ci.City_Id
GROUP BY ve.Venue_Id, ve.Venue_Name,ci.City_Name
ORDER BY COUNT(ma.Outcome_type) DESC;
"""
No_of_match_played=pd.read_sql(sql,conn)
No_of_match_played.head(5).style.background_gradient(subset=["No_of_match_played"], cmap='Blues',low=0.55)

,Venue_Id,Venue_Name,No_of_match_played,Location
0,1,M Chinnaswamy Stadium,58,Bangalore
1,5,Eden Gardens,54,Kolkata
2,3,Feroz Shah Kotla,53,Delhi
3,4,Wankhede Stadium,49,Mumbai
4,8,"MA Chidambaram Stadium, Chepauk",48,Chennai


In [6]:
#2.What is the average number of runs scored in each season?
sql=""" SELECT se.Season_Id,se.Season_Year,ROUND(AVG(ba.Runs_Scored)*100) AS AVG_Runs
FROM Batsman_Scored ba
JOIN Match me 
ON me.Match_Id=ba.Match_Id
JOIN Season se
ON me.Season_Id=se.Season_Id
GROUP BY se.Season_Id,se.Season_Year
ORDER BY se.Season_Year DESC;
"""
AVG_Runs=pd.read_sql(sql,conn)
AVG_Runs.style.background_gradient(subset=["AVG_Runs"], cmap='Blues_r',low=0.55)


,Season_Id,Season_Year,AVG_Runs
0,9,2016,131.000000
1,8,2015,132.000000
2,7,2014,129.000000
3,6,2013,120.000000
4,5,2012,120.000000
5,4,2011,121.000000
6,3,2010,127.000000
7,2,2009,117.000000
8,1,2008,129.000000


In [7]:
#3.Number of matches played in each city¶
sql=""" SELECT ce.City_Name,SUM(me.Outcome_type) AS No_Of_Matches
FROM City ce
JOIN Venue ve ON ve.City_Id=ce.City_Id
JOIN Match me ON me.Venue_Id=ve.Venue_Id
GROUP BY ce.City_Name
ORDER BY COUNT(me.Outcome_type) DESC;
"""
No_Of_Matches=pd.read_sql(sql,conn)
No_Of_Matches.head().style.background_gradient(subset=["No_Of_Matches"], cmap='Blues',low=0.55)

,City_Name,No_Of_Matches
0,Mumbai,77
1,Bangalore,62
2,Kolkata,54
3,Delhi,54
4,Chennai,50


In [8]:
#4.Top 10 wicket takers¶
sql = """
select pe.player_name, COUNT(wt.Kind_Out) Total_Wickets
from Wicket_taken wt
JOIN Ball_by_ball bl
ON wt.Match_id = bl.Match_Id
   AND wt.Over_id = bl.Over_id
   AND wt.ball_id = bl.ball_id
   AND wt.Innings_No = bl.Innings_No
JOIN player pe
ON pe.player_id = bl.bowler

Group by bl.bowler
ORDER BY COUNT(wt.Kind_Out) DESC
LIMIT 10
"""
wicket_takers = pd.read_sql(sql, conn)
wicket_takers.style.background_gradient(subset=["Total_Wickets"], cmap='Greens',low=0.55)

,Player_Name,Total_Wickets
0,SL Malinga,159
1,DJ Bravo,137
2,A Mishra,132
3,Harbhajan Singh,128
4,PP Chawla,127
5,R Vinay Kumar,123
6,A Nehra,111
7,R Ashwin,110
8,Z Khan,107
9,DW Steyn,100


In [9]:
#5.No of players in the IPL country wise
sql="""
SELECT co.Country_Id,co.Country_Name,COUNT(pl.Player_Name) as No_Of_Players_country_wise
FROM Player pl
JOIN Country co
ON co.Country_Id=pl.Country_Name
GROUP BY co.Country_Id,co.Country_Name
ORDER BY SUM(pl.Player_Name) DESC;
"""
No_Of_Players=pd.read_sql(sql,conn)
No_Of_Players.style.background_gradient(subset=["No_Of_Players_country_wise"], cmap='Greys',low=0.55)

,Country_Id,Country_Name,No_Of_Players_country_wise
0,1,India,262
1,2,South Africa,39
2,4,New Zealand,22
3,5,Australia,72
4,6,Pakistan,13
5,7,Sri Lanka,20
6,8,West Indies,19
7,9,Zimbabwea,2
8,10,England,14
9,11,Bangladesh,5


In [10]:
#6.Which bowling style has the highest number of wickets
sql="""
SELECT bo.Bowling_skill,COUNT(wt.Kind_Out) AS highest_number_of_wickets
FROM Wicket_Taken wt 
JOIN Ball_by_Ball ba
ON wt.Match_Id=ba.Match_Id
AND wt.Over_Id=ba.Over_Id
AND wt.Ball_Id=ba.Ball_Id
AND wt.Innings_No=ba.Innings_No
JOIN Bowling_Style bo
ON bo.Bowling_Id=ba.Bowler
GROUP BY bo.Bowling_Id
ORDER BY COUNT(wt.Kind_Out) DESC;
"""
highest_number_of_wickets=pd.read_sql(sql,conn)
highest_number_of_wickets.style.background_gradient(subset=["highest_number_of_wickets"],cmap="cubehelix",low=0.25)

,Bowling_skill,highest_number_of_wickets
0,Left-arm fast,96
1,Left-arm medium-fast,74
2,Right-arm medium,12
3,Legbreak googly,10
4,Right-arm bowler,7
5,Slow left-arm chinaman,5
6,Right-arm medium-fast,2
7,Left-arm medium,1
8,Legbreak,1


In [11]:
#7.Number matches won by each team per year 
sql = """
SELECT  te.Team_Name as Team_Name, se.Season_Year, Count(ma.Match_Winner) AS Number_of_wins
FROM Match ma
JOIN Season se
ON ma.Season_Id=se.Season_Id
JOIN Team te
ON te.Team_Id  = ma.Match_Winner
GROUP BY te.Team_Name,se.Season_Year
ORDER BY Count(ma.Match_Winner) DESC
LIMIT 10
"""
Number_of_wins = pd.read_sql(sql, conn)
Number_of_wins.style.background_gradient(subset=["Number_of_wins"],cmap="copper",low=0.25)

,Team_Name,Season_Year,Number_of_wins
0,Mumbai Indians,2013,13
1,Rajasthan Royals,2008,13
2,Chennai Super Kings,2013,12
3,Kings XI Punjab,2014,12
4,Kolkata Knight Riders,2012,12
5,Chennai Super Kings,2011,11
6,Delhi Daredevils,2012,11
7,Kolkata Knight Riders,2014,11
8,Mumbai Indians,2010,11
9,Rajasthan Royals,2013,11


In [12]:
#8.Total matches won by each team
sql = """
SELECT  te.Team_Name as Team_Name,Count(ma.Match_Winner) AS Matches
FROM Match ma
JOIN Team te
ON te.Team_Id  = ma.Match_Winner
GROUP BY te.Team_Name
ORDER BY Count(ma.Match_Winner) DESC
LIMIT 10
"""
Matches = pd.read_sql(sql, conn)
Matches.style.background_gradient(subset=["Matches"],cmap="summer",low=0.25)

,Team_Name,Matches
0,Mumbai Indians,80
1,Chennai Super Kings,79
2,Royal Challengers Bangalore,70
3,Kolkata Knight Riders,68
4,Rajasthan Royals,63
5,Kings XI Punjab,63
6,Delhi Daredevils,56
7,Sunrisers Hyderabad,34
8,Deccan Chargers,29
9,Pune Warriors,12


In [13]:
#9.Highest Wicket takers in IPL
sql=""" select Player_Name,count(Kind_Out) as Wickets,
        sum(case when Kind_Out==1 then 1 else 0 end) as caught,
        sum(case when Kind_Out==2 then 1 else 0 end) as bowled,
        sum(case when Kind_Out==4 then 1 else 0 end) as lbw,
        sum(case when Kind_Out==6 then 1 else 0 end) as stumped
        from Wicket_Taken a
        join Ball_by_Ball b
        on a.Match_Id=b.Match_Id 
        and a.Innings_No=b.Innings_No
        and a.Over_Id=b.Over_Id 
        and a.Ball_Id=b.Ball_Id
        join (select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Bowling_Style c
              on a.Bowling_skill=c.Bowling_Id) c
        on b.Bowler=c.Player_Id
        where Kind_Out in (1,2,4,6,7,8)
        group by c.Player_Id
        order by Wickets desc; """

wickets=pd.read_sql(sql, conn)
subset=["Wickets","caught","bowled","lbw","stumped","caught and bowled"]
wickets.head().style.background_gradient(subset=["Wickets"],cmap="binary",low=0.25)

,Player_Name,Wickets,caught,bowled,lbw,stumped
0,SL Malinga,143,69,59,10,0
1,A Mishra,124,66,21,13,21
2,DJ Bravo,122,90,18,6,0
3,PP Chawla,120,56,33,14,13
4,Harbhajan Singh,119,61,25,8,14


In [14]:
#10.Top 5 Run scorers¶
sql = """
select p.player_id,p.player_name,SUM(ba.Runs_Scored) as Total_Runs
from batsman_scored ba

JOIN Ball_by_ball bl
ON ba.Match_id = bl.Match_Id
   AND ba.Over_id = bl.Over_id
   AND ba.ball_id = bl.ball_id
   AND ba.Innings_No = bl.Innings_No
JOIN player p
ON p.player_id = bl.striker
Group by bl.Striker
ORDER BY SUM(ba.Runs_Scored) DESC
LIMIT 10
"""
runs_scorers = pd.read_sql(sql, conn)
runs_scorers.head(5).style.background_gradient(subset=["Total_Runs"], cmap='Oranges',low=0.55)

,Player_Id,Player_Name,Total_Runs
0,21,SK Raina,4106
1,8,V Kohli,4105
2,57,RG Sharma,3874
3,40,G Gambhir,3634
4,162,CH Gayle,3447


In [15]:
#11.Wicket types
sql="""
SELECT ot.Out_Name,COUNT(wt.Kind_Out) AS Wicket_types
FROM Wicket_Taken wt 
JOIN Out_Type ot
ON wt.Kind_Out=ot.Out_Id
GROUP BY ot.Out_Name
ORDER BY COUNT(wt.Kind_Out) DESC
"""
Wicket_types=pd.read_sql(sql,conn)
Wicket_types.style.background_gradient(subset=["Wicket_types"],cmap="cubehelix",low=0.25)

,Out_Name,Wicket_types
0,caught,3954
1,bowled,1251
2,run out,697
3,lbw,399
4,stumped,222
5,caught and bowled,187
6,retired hurt,8
7,hit wicket,8
8,obstructing the field,1


In [16]:
#12. Man of the Series, Purple Cap, Orange Cap winners in every season
sql="""
SELECT se.Season_Year,mos.Player_Name Man_of_the_Series,oc.Player_Name Orange_Cap,pc.Player_Name Purple_Cap
FROM Season se
JOIN Player mos ON mos.Player_Id=se.Man_of_the_Series
JOIN Player oc ON oc.Player_Id=se.Orange_Cap
JOIN Player pc ON pc.Player_Id=se.Purple_Cap
"""
Query=pd.read_sql(sql,conn)
Query.style.background_gradient(subset=["Season_Year"],cmap="summer",low=0.25)

,Season_Year,Man_of_the_Series,Orange_Cap,Purple_Cap
0,2008,SR Watson,SE Marsh,Sohail Tanvir
1,2009,AC Gilchrist,ML Hayden,RP Singh
2,2010,SR Tendulkar,SR Tendulkar,PP Ojha
3,2011,CH Gayle,CH Gayle,SL Malinga
4,2012,SP Narine,CH Gayle,M Morkel
5,2013,SR Watson,MEK Hussey,DJ Bravo
6,2014,GJ Maxwell,RV Uthappa,MM Sharma
7,2015,AD Russell,DA Warner,DJ Bravo
8,2016,V Kohli,V Kohli,B Kumar


In [17]:
 #13.total extras (wides, no-balls, etc.) conceded by each team in a specific season
sql="""
SELECT se.Season_Year,te.Team_Name,et.Extra_Name,COUNT(ex.Extra_Runs) AS Total_Extras
FROM Extra_Runs ex
JOIN Extra_Type et
ON ex.Extra_Type_Id=et.Extra_Id
JOIN Season se
JOIN Team te
GROUP BY se.Season_Year,te.Team_Name,et.Extra_Name
ORDER BY COUNT(ex.Extra_Runs) DESC;
"""
Total_Extras=pd.read_sql(sql,conn)
Total_Extras.sample(10).style.background_gradient(subset=["Total_Extras"],cmap="summer",low=0.25)

,Season_Year,Team_Name,Extra_Name,Total_Extras
249,2009,Delhi Daredevils,noballs,579
46,2011,Mumbai Indians,wides,4153
169,2012,Chennai Super Kings,legbyes,2357
391,2011,Deccan Chargers,byes,379
489,2009,Pune Warriors,penalty,1
395,2011,Kochi Tuskers Kerala,byes,379
202,2014,Mumbai Indians,legbyes,2357
402,2011,Sunrisers Hyderabad,byes,379
138,2009,Pune Warriors,legbyes,2357
533,2013,Chennai Super Kings,penalty,1


In [18]:
#14.the percentage of matches won by teams that choose to bat first after winning the toss.
sql="""SELECT
    T.Team_Name,
    COUNT(M.Match_Id) AS Total_Matches,
    SUM(CASE WHEN Toss.Toss_Name = 'bat' AND M.Match_Winner = T.Team_Id THEN 1 ELSE 0 END) AS Matches_Won,
    (SUM(CASE WHEN Toss.Toss_Name = 'bat' AND M.Match_Winner = T.Team_Id THEN 1.0 ELSE 0 END) / COUNT(M.Match_Id)) * 100 AS WinningPercentage
FROM
    Match AS M
JOIN
    Team AS T
ON
    M.Match_Winner = T.Team_Id
JOIN
    Toss_Decision AS Toss
ON
    M.Toss_Decide = Toss.Toss_Id
GROUP BY
    T.Team_Name
ORDER BY
    WinningPercentage DESC
    LIMIT 10

"""
Percentage_Won = pd.read_sql(sql, conn)
Percentage_Won.style.background_gradient(subset=["WinningPercentage"],cmap="Oranges",low=0.25)

,Team_Name,Total_Matches,Matches_Won,WinningPercentage
0,Pune Warriors,12,9,75.000000
1,Chennai Super Kings,79,50,63.291139
2,Kolkata Knight Riders,68,33,48.529412
3,Deccan Chargers,29,14,48.275862
4,Rajasthan Royals,63,30,47.619048
5,Mumbai Indians,80,38,47.500000
6,Delhi Daredevils,56,26,46.428571
7,Sunrisers Hyderabad,34,14,41.176471
8,Rising Pune Supergiants,5,2,40.000000
9,Royal Challengers Bangalore,70,24,34.285714


In [27]:
#15.Players Who Played for Multiple Teams in Different Seasons:
sql="""
SELECT pl.Player_Name,GROUP_CONCAT(te.team_name) AS teams_played_for,se.Season_Year
FROM Team te
JOIN Match ma
ON te.Team_Id=ma.Match_Winner
JOIN Player pl
ON pl.Player_Id=ma.Man_of_the_Match
JOIN Season se 
ON se.Season_Id=ma.Season_Id
GROUP BY pl.Player_Name,se.Season_Year
ORDER BY GROUP_CONCAT(te.team_name)
"""

teams_played_for=pd.read_sql(sql,conn)
teams_played_for.sample(10)


,Player_Name,teams_played_for,Season_Year
340,R Dravid,Royal Challengers Bangalore,2009
344,RV Uthappa,Royal Challengers Bangalore,2010
355,JH Kallis,"Royal Challengers Bangalore,Royal Challengers ...",2009
45,DR Smith,"Chennai Super Kings,Chennai Super Kings,Chenna...",2014
168,SE Marsh,"Kings XI Punjab,Kings XI Punjab,Kings XI Punja...",2008
134,BA Bhatt,Kings XI Punjab,2011
37,MS Dhoni,"Chennai Super Kings,Chennai Super Kings",2008
125,AC Gilchrist,Kings XI Punjab,2012
26,SK Raina,Chennai Super Kings,2011
203,YK Pathan,Kolkata Knight Riders,2014


In [29]:
#16.No of fifties and centruies by a batsman in an IPL

sql=""" select Player_Name,Country_Name,
        sum(case when runs>=50 and runs<100 then 1 else 0 end) as fifties,
        sum(case when runs>=100 then 1 else 0 end) as centuries,
        max(runs) as highest_score
        from(select Striker,sum(Runs_Scored) as runs
             from Ball_by_Ball a
             join Batsman_Scored b
             on a.Match_Id=b.Match_Id 
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id 
             and a.Ball_Id=b.Ball_Id
             group by Striker,a.Match_Id
             having runs>=50) a
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
             from Player a
             join Country b
             on a.Country_Name=b.Country_Id
             join Batting_Style c
             on a.Batting_hand=c.Batting_Id) b
        on a.Striker=b.Player_Id
        group by Player_Id
        order by highest_score desc; """

best_player=pd.read_sql(sql, conn)
best_player.head(10).style.background_gradient(subset=["fifties","centuries","highest_score"],low=0.55,axis=0)

,Player_Name,Country_Name,fifties,centuries,highest_score
0,CH Gayle,West Indies,21,5,175
1,BB McCullum,New Zealand,11,2,158
2,AB de Villiers,South Africa,22,3,133
3,M Vijay,India,13,2,127
4,V Sehwag,India,16,2,122
5,PC Valthaty,India,2,1,120
6,A Symonds,Australia,5,1,117
7,MEK Hussey,Australia,15,1,116
8,SE Marsh,Australia,18,1,115
9,WP Saha,India,5,1,115


In [31]:
#17.Numer of dissmissals by each wicket type
sql = """
select o.Out_Name AS Wicket_type, COUNT(w.Kind_Out) AS Total_dissmissals
from Out_Type o
JOIN Wicket_Taken w
ON o.Out_Id = w.Kind_Out
GROUP BY 1
ORDER BY Total_dissmissals DESC
"""
wickets = pd.read_sql(sql, conn)
wickets.style.background_gradient(subset=["Total_dissmissals"],cmap="Oranges",low=0.25)

,Wicket_type,Total_dissmissals
0,caught,3954
1,bowled,1251
2,run out,697
3,lbw,399
4,stumped,222
5,caught and bowled,187
6,retired hurt,8
7,hit wicket,8
8,obstructing the field,1


In [40]:
#18. Total Number of Wickets Taken in Each Over

sql="""
SELECT Over_Id AS Over_Numbers, COUNT(Over_Id) As Wickets
FROM Wicket_Taken
GROUP BY Over_Id
ORDER BY COUNT(Over_Id) DESC
LIMIT 10
"""
Wickets=pd.read_sql(sql,conn)
Wickets.style.background_gradient(subset=["Wickets"],cmap="summer",low=0.25)

,Over_Numbers,Wickets
0,20,678
1,19,538
2,18,506
3,17,411
4,16,375
5,15,340
6,14,317
7,11,300
8,3,296
9,5,295


In [122]:
#19.Win-Loss Statistics
sql = """
SELECT te.Team_Name,
       COUNT(ma.Match_Id) AS Total_Matches,
       SUM(CASE WHEN te.Team_Id = ma.Match_Winner THEN 1 ELSE 0 END) AS Wins,
       SUM(CASE WHEN te.Team_Id <> ma.Match_Winner THEN 1 ELSE 0 END) AS Losses,
       ROUND(SUM(CASE WHEN te.Team_Id = ma.Match_Winner THEN 1 ELSE 0 END)* 100.0 / COUNT(ma.Match_Id), 0) AS Wins_Percentage,       
       ROUND(SUM(CASE WHEN te.Team_Id <> ma.Match_Winner THEN 1 ELSE 0 END) * 100.0 /COUNT(ma.Match_Id), 0) AS Loss_Percentage
FROM Team te
JOIN Match ma 
ON te.Team_Id = ma.Match_Winner 
OR te.Team_Id = ma.Team_1 
OR te.Team_Id = ma.Team_2
GROUP BY te.Team_Name
ORDER BY Total_Matches DESC
LIMIT 10

"""
win_loss=pd.read_sql(sql,conn)
win_loss.style.background_gradient(subset=["Wins","Losses","Wins_Percentage","Loss_Percentage"],cmap="Blues",low=0.25)

,Team_Name,Total_Matches,Wins,Losses,Wins_Percentage,Loss_Percentage
0,Mumbai Indians,140,80,60,57.000000,43.000000
1,Royal Challengers Bangalore,139,70,67,50.000000,48.000000
2,Kings XI Punjab,134,63,71,47.000000,53.000000
3,Delhi Daredevils,133,56,75,42.000000,56.000000
4,Kolkata Knight Riders,132,68,64,52.000000,48.000000
5,Chennai Super Kings,131,79,52,60.000000,40.000000
6,Rajasthan Royals,118,63,54,53.000000,46.000000
7,Deccan Chargers,75,29,46,39.000000,61.000000
8,Sunrisers Hyderabad,62,34,28,55.000000,45.000000
9,Pune Warriors,46,12,33,26.000000,72.000000


In [123]:
#20.Bowlers who have bowled most deliveries

sql="""SELECT Player_Name,c.Country_Name,d.Bowling_skill,count(*) as Deliveries
       FROM Ball_by_Ball a
       join Player b
       on a.Bowler==b.Player_Id
       join Country c
       on b.Country_Name==c.Country_Id
       join Bowling_Style d
       on b.Bowling_skill==d.Bowling_Id
       group by Bowler
       order by Deliveries desc ;"""

bowling=pd.read_sql(sql, conn)
bowling.head().style.background_gradient(subset=["Deliveries"],cmap="Reds",low=0.25)

,Player_Name,Country_Name,Bowling_skill,Deliveries
0,Harbhajan Singh,India,Right-arm offbreak,2742
1,P Kumar,India,Right-arm medium,2529
2,PP Chawla,India,Legbreak,2472
3,A Mishra,India,Legbreak,2466
4,SL Malinga,Sri Lanka,Right-arm fast,2407
